# Yield calculator

This notebook is used to estimate the CCSNe yields consistent with the present-day APOGEE trend in \[C/Mg\]-\[Mg/H\].

uses the results from the MCMC fits to AGB integrated yields with 

In [13]:
import toml
import surp
import vice

import vice.yields.ccsne.settings as y_cc
import vice.yields.agb.settings as Y_agb
import vice.yields.sneia.settings as y_ia

In [14]:
surp.set_yields(verbose=False)

# yield params calculator

Given the desired f_agb or alpha, what are the required y0cc and alpha_agb values

## inputs from mcmc fits

In [3]:
Y_C_AGB = {
    "cristallo11": 3.8e-4,
    "ventura13": 1.85e-4,
    "karakas16": 2.8e-4,
    "pignatari16": 5.9e-4
}

In [4]:
ZETA_C_AGB = {
    "cristallo11": -3.5e-4,
    "ventura13": -9.4e-4,
    "karakas16": -10.1e-4,
    "pignatari16": -5.7e-4
}

In [5]:
y_mg = y_cc["mg"]
noneq_factor = 1.025
noneq_zeta = 1.8
noneq_A = 1.8

Y_C_0 = 4.12 * y_mg * noneq_factor
ZETA_C_0 = 1.21 * y_mg * noneq_factor * noneq_zeta
A_C_0 = 3.07 * y_mg * noneq_factor * noneq_A

In [6]:
Y_C_0 * 0.2

0.0005506792000000001

## Utils

In [7]:
def f_agb_to_alpha(y0_agb, f_agb):
    y_agb = Y_C_0 * f_agb
    return y_agb / y0_agb

In [8]:
def f_agb_to_settings(study, y0=None, f_agb=0.2):
    if y0 is None:
        y0 = Y_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(toml.dumps(settings))

In [9]:
def f_agb_zeta_to_settings(study, *, y0_agb=None, zeta_agb=None, f_agb=0.2):
    if y0_agb is None:
        y0_agb = Y_C_AGB[study]
    if zeta_agb is None:
        zeta_agb = ZETA_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0_agb, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    
    zeta_cc = ZETA_C_0 - zeta_agb * alpha
    
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc, zeta_c_cc=zeta_cc)
    
    print(toml.dumps(settings))

In [10]:
def alpha_to_settings(study, alpha):
    f_agb =  alpha * Y_C_AGB[study] / Y_C_0
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(toml.dumps(settings))

In [11]:
def enhance_fe_ia(factor):
    """if factor != 1, then enhances sne ia fe yield by factor but maintains same total fe yield, applied to params"""
    if factor == 1:
        return
    y_fe = y_ia["fe"] + y_cc["fe"]
    y_fe_ia = y_ia["fe"] * factor
    y_fe_cc = y_fe - y_fe_ia
    
    print(f"y_fe_ia = {y_fe_ia:0.8f}")
    print(f"y_fe_cc = {y_fe_cc:0.8f}")
    return y_fe_ia, y_fe_cc


In [12]:
def m_factor_to_settings(Y_c_agb="cristallo11"):
    surp.set_yields(y_c_cc=0, Y_c_agb=Y_c_agb, kwargs_c_agb = dict(mass_factor=m), verbose=False)
    y_agb = surp.yields.calc_y(surp.Z_SUN)
    
    print(y_cc_0 - surp.yields.calc_y(surp.Z_SUN) + y_agb_0)

## Fruity

In [34]:
alpha_to_settings("cristallo11", alpha=1.449)

alpha_c_agb = 1.449
y0_c_cc = 0.002202776



In [16]:
enhance_fe_ia(1.2)

y_fe_ia = 0.00092400
y_fe_cc = 0.00031900


(0.0009239999999999999, 0.00031900000000000006)

In [15]:
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.8, 1]:
    print("f_agb = ", f)
    f_agb_zeta_to_settings("cristallo11", f_agb=f)
    print()

f_agb =  0
alpha_c_agb = 0.0
y0_c_cc = 0.002753396
zeta_c_cc = 0.0014555574


f_agb =  0.1
alpha_c_agb = 0.7245778947368421
y0_c_cc = 0.0024780564
zeta_c_cc = 0.0017091596631578947


f_agb =  0.2
alpha_c_agb = 1.4491557894736842
y0_c_cc = 0.0022027168000000002
zeta_c_cc = 0.0019627619263157896


f_agb =  0.3
alpha_c_agb = 2.173733684210526
y0_c_cc = 0.0019273772
zeta_c_cc = 0.0022163641894736843


f_agb =  0.4
alpha_c_agb = 2.8983115789473683
y0_c_cc = 0.0016520375999999999
zeta_c_cc = 0.002469966452631579


f_agb =  0.5
alpha_c_agb = 3.6228894736842103
y0_c_cc = 0.001376698
zeta_c_cc = 0.0027235687157894737


f_agb =  0.8
alpha_c_agb = 5.796623157894737
y0_c_cc = 0.0005506791999999998
zeta_c_cc = 0.003484375505263158


f_agb =  1
alpha_c_agb = 7.245778947368421
y0_c_cc = 0.0
zeta_c_cc = 0.003991580031578948




## ATON

In [29]:
f_agb_zeta_to_settings("ventura13", f_agb=0.2)


alpha_c_agb = 2.9766443243243246
y0_c_cc = 0.0022027168000000002
zeta_c_cc = 0.004253603064864865



In [36]:
alpha_to_settings("ventura13", alpha=2)
print()
alpha_to_settings("ventura13", alpha=3)

alpha_c_agb = 2
y0_c_cc = 0.002383396


alpha_c_agb = 3
y0_c_cc = 0.002198396



## Monash

In [37]:
f_agb_zeta_to_settings("karakas16", f_agb=0.2)


alpha_c_agb = 1.9667114285714289
y0_c_cc = 0.0022027168000000002
zeta_c_cc = 0.003441935942857143



In [15]:
for f in [0, 0.1, 0.3, 0.4, 0.5, 1]:
    print(f)
    f_agb_zeta_to_settings("karakas16", f_agb=f)

0
alpha_c_agb = 0.0
y0_c_cc = 0.002753396
zeta_c_cc = 0.0014555574

0.1
alpha_c_agb = 0.9833557142857144
y0_c_cc = 0.0024780564
zeta_c_cc = 0.0024487466714285715

0.3
alpha_c_agb = 2.9500671428571428
y0_c_cc = 0.0019273772
zeta_c_cc = 0.004435125214285714

0.4
alpha_c_agb = 3.9334228571428578
y0_c_cc = 0.0016520375999999999
zeta_c_cc = 0.0054283144857142866

0.5
alpha_c_agb = 4.916778571428572
y0_c_cc = 0.001376698
zeta_c_cc = 0.006421503757142857

1
alpha_c_agb = 9.833557142857144
y0_c_cc = 0.0
zeta_c_cc = 0.011387450114285715



## NuGrid

In [31]:
f_agb_zeta_to_settings("pignatari16", f_agb=0.2)

alpha_c_agb = 0.9333545762711865
y0_c_cc = 0.0022027168000000002
zeta_c_cc = 0.0019875695084745765



In [14]:
for f in [0, 0.1, 0.3, 0.5, 1]:
    f_agb_zeta_to_settings("pignatari16", f_agb=f)

alpha_c_agb = 0.0
y0_c_cc = 0.002753396
zeta_c_cc = 0.0014555574

alpha_c_agb = 0.4666772881355932
y0_c_cc = 0.0024780564
zeta_c_cc = 0.0017215634542372881

alpha_c_agb = 1.4000318644067795
y0_c_cc = 0.0019273772
zeta_c_cc = 0.0022535755627118644

alpha_c_agb = 2.333386440677966
y0_c_cc = 0.001376698
zeta_c_cc = 0.0027855876711864406

alpha_c_agb = 4.666772881355932
y0_c_cc = 0.0
zeta_c_cc = 0.004115617942372881



### Mass factor

In [72]:
for m in [0.5, 0.8, 1, 1.5, 2]:
    surp.set_yields(y_c_cc=0, kwargs_c_agb = dict(mass_factor=m), verbose=False)
    print(m)
    print(surp.yields.calc_y(surp.Z_SUN))
    print(y_cc_0 - surp.yields.calc_y(surp.Z_SUN) + y_agb_0)

0.5
0.000420468442011002
0.0022502165963843614
0.8
0.0005204255132192604
0.002150259525176103
1
0.00046798503839536314
0.0022027
1.5
0.00047631938906597107
0.002194365649329392
2
8.347695469561248e-05
0.002587208083699751


In [71]:
y_agb_0 = 0.00046798503839536314
y_cc_0 = 0.0022027

In [72]:
for m in [0.5, 0.8, 1, 1.5, 2]:
   

0.5
0.000420468442011002
0.0022502165963843614
0.8
0.0005204255132192604
0.002150259525176103
1
0.00046798503839536314
0.0022027
1.5
0.00047631938906597107
0.002194365649329392
2
8.347695469561248e-05
0.002587208083699751


## Analytic

In [8]:
y0_agb_a = Y_C_AGB["cristallo11"]
y0_agb_a

0.00038

In [9]:
zeta_agb_a = ZETA_C_AGB["cristallo11"]
zeta_agb_a

-0.00035

In [10]:
def get_alpha(f_agb=0.2, y0_agb=y0_agb_a):
    return Y_C_0 / y0_agb * f_agb

In [11]:
def get_zeta_cc(alpha, zeta=zeta_agb_a):
    return ZETA_C_0 - alpha*zeta

In [12]:
def get_ana_settings(f, zeta=zeta_agb_a):
    alpha = get_alpha(f)
    print("alpha_c_agb = ", alpha)
    print("y0_c_cc = ", Y_C_0 - alpha * y0_agb_a)
    print("zeta_c_cc = ", get_zeta_cc(alpha, zeta))

In [13]:
get_ana_settings(0.2)

alpha_c_agb =  1.4491557894736842
y0_c_cc =  0.0022027168
zeta_c_cc =  0.0019627619263157896


In [14]:
get_ana_settings(0.2, 0)

alpha_c_agb =  1.4491557894736842
y0_c_cc =  0.0022027168
zeta_c_cc =  0.0014555574


In [15]:
get_ana_settings(0.2, -0.0007)

alpha_c_agb =  1.4491557894736842
y0_c_cc =  0.0022027168
zeta_c_cc =  0.002469966452631579


In [16]:
for f in [0, 0.1, 0.3, 0.5, 1]:
    print(f)
    get_ana_settings(f)
    print()

0
alpha_c_agb =  0.0
y0_c_cc =  0.002753396
zeta_c_cc =  0.0014555574

0.1
alpha_c_agb =  0.7245778947368421
y0_c_cc =  0.0024780564
zeta_c_cc =  0.0017091596631578947

0.3
alpha_c_agb =  2.173733684210526
y0_c_cc =  0.0019273772000000002
zeta_c_cc =  0.0022163641894736843

0.5
alpha_c_agb =  3.6228894736842103
y0_c_cc =  0.001376698
zeta_c_cc =  0.0027235687157894737

1
alpha_c_agb =  7.245778947368421
y0_c_cc =  0.0
zeta_c_cc =  0.003991580031578948



In [17]:
y = surp.yield_models.C_AGB_Model()

In [18]:
print(y)

(4.00e-04 + -2.00e-04(Z-Zo)), t_D=0.15, tau=0.30


In [19]:
print(2*y)

(8.00e-04 + -4.00e-04(Z-Zo)), t_D=0.15, tau=0.30


In [37]:
print(0.15 * y)

(6.00e-05 + -3.00e-05(Z-Zo)), t_D=0.15, tau=0.30
